In [25]:
from qiskit import QuantumCircuit, transpile, QuantumRegister
from qiskit.providers.fake_provider import FakeQuito
from qiskit.circuit import Qubit
from qiskit.circuit import Parameter
from qiskit import Aer
import numpy as np

In [26]:
def calc_expected_fidelity_ibm(qc: QuantumCircuit, calibration) -> float:
    res = 1.0
    for instruction, qargs, _cargs in qc.data:
        gate_type = instruction.name

        assert gate_type in ["rz", "sx", "x", "cx", "measure", "barrier"]

        if gate_type != "barrier":
            assert len(qargs) in [1, 2]
            first_qubit = calc_qubit_index(qargs, qc.qregs, 0)
            if len(qargs) == 1:

                if gate_type == "measure":
                    specific_error: float = calibration.readout_error(first_qubit)
                else:
                    specific_error = calibration.gate_error(gate_type, [first_qubit])

            else:
                second_qubit = calc_qubit_index(qargs, qc.qregs, 1)

                specific_error = calibration.gate_error(gate_type, [first_qubit, second_qubit])
                    

            res *= 1 - specific_error
    return res

def calc_qubit_index(qargs:list[Qubit], qregs: list[QuantumRegister], index: int) -> int:
    offset = 0
    for reg in qregs:
        if qargs[index] not in reg:
            offset += reg.size
        else:
            qubit_index: int = offset + reg.index(qargs[index])
            return qubit_index
    error_msg = f"Global qubit index for local qubit {index} index not found."
    raise ValueError(error_msg)

In [27]:
basis_gates_ibm = ['rz', 'x', 'sx', 'cx', 'id']
qc = QuantumCircuit(4)
qc.h([0,1])
qc.cx(0,2)
qc.cx(1,3)
qc.rz(Parameter("x"), [0,1,2,3])
qc.cx(0,1)
qc.cx(2,3)
qc.measure_all()
qc.draw(fold=-1)


┌───┐     ┌───────┐               ░ ┌─┐         
   q_0: ┤ H ├──■──┤ Rz(x) ├───────────■───░─┤M├─────────
        ├───┤  │  └───────┘┌───────┐┌─┴─┐ ░ └╥┘┌─┐      
   q_1: ┤ H ├──┼──────■────┤ Rz(x) ├┤ X ├─░──╫─┤M├──────
        └───┘┌─┴─┐    │    ├───────┤└───┘ ░  ║ └╥┘┌─┐   
   q_2: ─────┤ X ├────┼────┤ Rz(x) ├──■───░──╫──╫─┤M├───
             └───┘  ┌─┴─┐  ├───────┤┌─┴─┐ ░  ║  ║ └╥┘┌─┐
   q_3: ────────────┤ X ├──┤ Rz(x) ├┤ X ├─░──╫──╫──╫─┤M├
                    └───┘  └───────┘└───┘ ░  ║  ║  ║ └╥┘
meas: 4/═════════════════════════════════════╩══╩══╩══╩═
                                             0  1  2  3

In [28]:
synthesized_qc = transpile(qc, basis_gates=basis_gates_ibm, optimization_level=3)
synthesized_qc.draw(fold=-1)

/Users/nils/Documents/repos/MQTPredictor/venv/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/nils/Documents/repos/MQTPredictor/venv/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


global phase: π/2
        ┌─────────┐┌────┐┌─────────┐     ┌───────┐               ░ ┌─┐         
   q_0: ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├──■──┤ Rz(x) ├───────────■───░─┤M├─────────
        ├─────────┤├────┤├─────────┤  │  └───────┘┌───────┐┌─┴─┐ ░ └╥┘┌─┐      
   q_1: ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├──┼──────■────┤ Rz(x) ├┤ X ├─░──╫─┤M├──────
        └─────────┘└────┘└─────────┘┌─┴─┐    │    ├───────┤└───┘ ░  ║ └╥┘┌─┐   
   q_2: ────────────────────────────┤ X ├────┼────┤ Rz(x) ├──■───░──╫──╫─┤M├───
                                    └───┘  ┌─┴─┐  ├───────┤┌─┴─┐ ░  ║  ║ └╥┘┌─┐
   q_3: ───────────────────────────────────┤ X ├──┤ Rz(x) ├┤ X ├─░──╫──╫──╫─┤M├
                                           └───┘  └───────┘└───┘ ░  ║  ║  ║ └╥┘
meas: 4/════════════════════════════════════════════════════════════╩══╩══╩══╩═
                                                                    0  1  2  3

In [29]:
backend = FakeQuito()
mapped_qc = transpile(synthesized_qc, basis_gates=basis_gates_ibm, optimization_level=3, coupling_map=backend.configuration().coupling_map, seed_transpiler=9)
mapped_qc.draw(fold=-1)

global phase: π/2
                                                                                            
ancilla_0 -> 0 ─────────────────────────────────────────────────────────────────────────────
                                           ┌───┐┌───────┐     ┌───┐           ░ ┌─┐         
      q_3 -> 1 ────────────────────────────┤ X ├┤ Rz(x) ├──■──┤ X ├──■────■───░─┤M├─────────
               ┌─────────┐┌────┐┌─────────┐└─┬─┘├───────┤  │  └─┬─┘  │  ┌─┴─┐ ░ └╥┘┌─┐      
      q_1 -> 2 ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├──■──┤ Rz(x) ├──┼────┼────┼──┤ X ├─░──╫─┤M├──────
               ├─────────┤├────┤├─────────┤     ├───────┤┌─┴─┐  │  ┌─┴─┐├───┤ ░  ║ └╥┘   ┌─┐
      q_0 -> 3 ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├──■──┤ Rz(x) ├┤ X ├──■──┤ X ├┤ X ├─░──╫──╫────┤M├
               └─────────┘└────┘└─────────┘┌─┴─┐├───────┤└───┘     └───┘└─┬─┘ ░  ║  ║ ┌─┐└╥┘
      q_2 -> 4 ────────────────────────────┤ X ├┤ Rz(x) ├─────────────────■───░──╫──╫─┤M├─╫─
                                           └───┘└───────┘                     ░  ║  ║ └╥┘ ║ 
       meas: 4/══════════════════════════════════════════════════════════════════╩══╩══╩══╩═
                                                                                 0  1  2  3

In [30]:
calc_expected_fidelity_ibm(mapped_qc, backend.properties())

0.5814435112777496

In [31]:
mapped_qc = mapped_qc.assign_parameters({mapped_qc.parameters[0]: np.pi})

In [32]:
mapped_qc.draw(fold=-1)

global phase: π/2
                                                                                            
ancilla_0 -> 0 ─────────────────────────────────────────────────────────────────────────────
                                           ┌───┐┌───────┐     ┌───┐           ░ ┌─┐         
      q_3 -> 1 ────────────────────────────┤ X ├┤ Rz(π) ├──■──┤ X ├──■────■───░─┤M├─────────
               ┌─────────┐┌────┐┌─────────┐└─┬─┘├───────┤  │  └─┬─┘  │  ┌─┴─┐ ░ └╥┘┌─┐      
      q_1 -> 2 ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├──■──┤ Rz(π) ├──┼────┼────┼──┤ X ├─░──╫─┤M├──────
               ├─────────┤├────┤├─────────┤     ├───────┤┌─┴─┐  │  ┌─┴─┐├───┤ ░  ║ └╥┘   ┌─┐
      q_0 -> 3 ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├──■──┤ Rz(π) ├┤ X ├──■──┤ X ├┤ X ├─░──╫──╫────┤M├
               └─────────┘└────┘└─────────┘┌─┴─┐├───────┤└───┘     └───┘└─┬─┘ ░  ║  ║ ┌─┐└╥┘
      q_2 -> 4 ────────────────────────────┤ X ├┤ Rz(π) ├─────────────────■───░──╫──╫─┤M├─╫─
                                           └───┘└───────┘                     ░  ║  ║ └╥┘ ║ 
       meas: 4/══════════════════════════════════════════════════════════════════╩══╩══╩══╩═
                                                                                 0  1  2  3

In [33]:
mapped_qc.draw(fold=-1)
backend = FakeQuito()
res_fake = backend.run(mapped_qc, shots=10000).result()
counts_fake = res_fake.get_counts(mapped_qc)
counts_fake

{'1001': 172,
 '0110': 150,
 '1110': 183,
 '1011': 188,
 '0100': 269,
 '1100': 239,
 '1000': 621,
 '0101': 1759,
 '0000': 2013,
 '0010': 352,
 '0111': 302,
 '1101': 513,
 '1111': 1332,
 '0001': 267,
 '0011': 138,
 '1010': 1502}

In [34]:
simulator = Aer.get_backend('aer_simulator')
result = simulator.run(mapped_qc, shots=10000).result()
counts = result.get_counts(mapped_qc)


In [35]:
def hist_intersection(original_counts: dict[str, int], current_counts: dict[str, int], ) -> float:
    all_keys = set(original_counts.keys()) | set(current_counts.keys())

    ideal_counts_filled = {key: original_counts.get(key, 0) for key in all_keys}
    counts_noisy_filled = {key: current_counts.get(key, 0) for key in all_keys}

    assert len(ideal_counts_filled.values()) == len(counts_noisy_filled.values())

    tmp_sum = 0
    for i in range(len(ideal_counts_filled.values())):
        tmp_sum += min(list(ideal_counts_filled.values())[i], list(counts_noisy_filled.values())[i])

    res = tmp_sum / sum(original_counts.values())
    print(res)
    return res

In [36]:
hist_intersection(counts, counts_fake)

0.6606


0.6606